In [8]:
import tkinter as tk
import math
import pandas as pd

In [9]:
#------------------------------------------------------
def nearest_tens(x_low, x_high):
    lower = math.floor(x_low / 10) * 10
    upper = math.ceil(x_high / 10) * 10
    if lower == x_low:
        lower -= 10
    if upper == x_high:
        upper += 10
    return lower, upper

#------------------------------------------------------
def get_data (file_name):
    data = pd.read_csv('data1.csv', header=None, names=['x', 'y', 'label'])
    
    x_min, x_max = data['x'].min(), data['x'].max()
    y_min, y_max = data['y'].min(), data['y'].max()
    unique_labels = data['label'].unique()
    return data, x_min,x_max,y_min,y_max,unique_labels #return data
#------------------------------------------------------


In [10]:
class Coordinate:
    # data = None
    # canvas = None
    # w = 0
    # h = 0
    # m = 0

    # value_x_h = 0
    # value_x_l = 0
    # value_y_h = 0
    # value_y_l = 0

    # coordinate_x_l = 0
    # coordinate_x_h = 0
    # coordinate_y_l = 0
    # coordinate_y_h = 0


    def __init__(self, width, height, margin, canvas):
        self.w = width
        self.h = height
        self.m = margin
        self.All_Points_Info = []
        self.canvas = canvas
        self.data, self.value_x_l, self.value_x_h, self.value_y_l, self.value_y_h, self.unique_labels = get_data('data2.csv')
        
        self.coordinate_x_l, self.coordinate_x_h = nearest_tens(self.value_x_l,self.value_x_h) #the lowest and highest value of x axis
        self.coordinate_y_l, self.coordinate_y_h = nearest_tens(self.value_y_l,self.value_y_h)


        print("Coordinate System Details:")
        print(f"  Width (w): {self.w}")
        print(f"  Height (h): {self.h}")
        print(f"  Margin (m): {self.m}")
        print(f"  X Axis Range: {self.coordinate_x_l} to {self.coordinate_x_h}")
        print(f"  Y Axis Range: {self.coordinate_y_l} to {self.coordinate_y_h}")
        print(f"  Value X Low (value_x_l): {self.value_x_l}")
        print(f"  Value X High (value_x_h): {self.value_x_h}")
        print(f"  Value Y Low (value_y_l): {self.value_y_l}")
        print(f"  Value Y High (value_y_h): {self.value_y_h}")
        print(f"  Number of Labels (number_labels): {self.unique_labels}")

    def create_axes_with_ticks(self):
        x_range = self.coordinate_x_h - self.coordinate_x_l
        y_range = self.coordinate_y_h - self.coordinate_y_l

        # Draw X and Y axes
        axis_color = 'black'
        self.canvas.create_line(2*self.m , self.h-2*self.m , 2*self.m , self.m ,arrow=tk.LAST, fill=axis_color) # X-axis
        self.canvas.create_line(2*self.m , self.h-2*self.m , self.w - self.m, self.h-2*self.m ,  arrow=tk.LAST,fill=axis_color) # Y-axis

        # Ticks settings
        tick_length = 5
        number_of_xticks = 10
        number_of_yticks = 10

        gap_x = (self.w-4*self.m) / number_of_xticks
        gap_y = (self.h-4*self.m) / number_of_yticks

        gap_x_value = x_range/number_of_xticks
        gap_y_value = y_range/number_of_yticks
        # Drawing X-axis ticks and labels
        for i in range(number_of_xticks + 1):
            x =  2*self.m +i*gap_x
            self.canvas.create_line(x, self.h - 2*self.m, x, self.h - 2*self.m - tick_length, fill=axis_color)
            self.canvas.create_text(x, self.h - 2*self.m + tick_length, text=str(self.coordinate_x_l + i*gap_x_value), anchor='n', font=('Arial', 8))

        # Drawing Y-axis ticks and labels
        for i in range(number_of_yticks + 1):
            y = self.h-2*self.m - i*gap_y
            self.canvas.create_line(2*self.m, y, 2*self.m + tick_length, y, fill=axis_color)
            self.canvas.create_text(2*self.m - tick_length, y, text=str(self.coordinate_y_l+i*gap_y_value), anchor='e', font=('Arial', 8))

        #legend
        legend_x_start = self.w - self.m - 70 # 图例开始的x坐标，这里我们将其设置在Canvas的右侧
        legend_y_start = 2  # 图例开始的y坐标，设置在Canvas的上方
        legend_gap = 20  # 图例中每项之间的间隔
        legend_icon_size = 10  # 图例图标的大小
        
        # 绘制图例背景
        self.canvas.create_rectangle(
    legend_x_start, 
    legend_y_start, 
    self.w - self.m, 
    legend_y_start + len(self.unique_labels) * legend_gap + 5,  # 减少了背景框的高度增量
    fill='white', 
    outline='black'
)

        
        # 遍历unique_labels，为每个标签绘制图例
        for i, label in enumerate(self.unique_labels):
            y_position = legend_y_start + i * legend_gap + legend_gap / 2
            # 根据标签决定图例的形状和颜色
            if label == self.unique_labels[0]:
                # 为第一个标签绘制红色圆形图例
                self.create_circle(legend_x_start + 10, y_position, 5, fill='red')
            elif label == self.unique_labels[1]:
                # 为第二个标签绘制蓝色星形图例
                self.draw_star(legend_x_start + 10, y_position, 5, fill='blue')
            elif label == self.unique_labels[2]:
                # 为第三个标签绘制灰色三角形图例
                self.create_triangle(legend_x_start + 10, y_position, 10, fill='grey')
            
            # 为每个图例项添加文本标签
            self.canvas.create_text(legend_x_start + 20, y_position, text=label, anchor="w", font=('Arial', 8))


    def translate(self,value_x,value_y):
        x_range = self.coordinate_x_h - self.coordinate_x_l
        y_range = self.coordinate_y_h - self.coordinate_y_l

        x_pixel = (value_x - self.value_x_l)/x_range*(self.w-4*self.m) + 2*self.m
        y_pixel = self.h- 2*self.m - (value_y - self.value_y_l)/y_range*(self.h-4*self.m)

        return x_pixel,y_pixel
    
    
    def create_circle(self, center_x, center_y, radius, **kwargs):
        x1 = center_x - radius
        y1 = center_y - radius
        x2 = center_x + radius
        y2 = center_y + radius
        self.canvas.create_oval(x1, y1, x2, y2, **kwargs)

    def draw_star(self,center_x, center_y, size, fill):
        points = []
        for i in range(5):
            # 计算外部顶点
            x = center_x + size * math.cos(2 * math.pi * i / 5 - math.pi / 10)
            y = center_y - size * math.sin(2 * math.pi * i / 5 - math.pi / 10)
            points.append((x, y))

            # 计算内部顶点
            x = center_x + size/2 * math.cos(2 * math.pi * i / 5 + math.pi / 10)
            y = center_y - size/2 * math.sin(2 * math.pi * i / 5 + math.pi / 10)
            points.append((x, y))

        self.canvas.create_polygon(points, outline='black', fill=fill)
    
    def create_triangle(self, center_x, center_y, size, **kwargs):

        height = size 
        side_length = height / (3**0.5 / 2)

        x1 = center_x
        y1 = center_y - 2 * height / 3

        x2 = center_x - side_length / 2
        y2 = center_y + height / 3

        x3 = center_x + side_length / 2
        y3 = center_y + height / 3

        self.canvas.create_polygon(x1, y1, x2, y2, x3, y3, **kwargs)

        

    def draw_points(self):
        for index, row in self.data.iterrows():
            x, y = self.translate(row['x'],row['y'])
            self.All_Points_Info.append({'value_x':row['x'],'value_y':row['y'],'pixel_x':x,'pixel_y':y,'label':row['label']})
            if row['label'] == self.unique_labels[0]:
                self.create_circle(x,y,5,fill = 'red')
            elif row['label'] == self.unique_labels[1]:
                self.draw_star(x,y,5,fill = 'blue')  
            elif row['label'] == self.unique_labels[2]:
                self.create_triangle(x,y,10,fill = 'grey')

    def get_All_Points_Info(self):
        return self.All_Points_Info


    # def check_click_near_point(self, click_x, click_y, threshold=5):
    #     for point in self.points_pixel:
    #         x, y, _ = point  # 假设点的格式为[x, y, label]
    #         distance = ((x - click_x) ** 2 + (y - click_y) ** 2) ** 0.5
    #         if distance <= threshold:
    #             return point  # 如果找到一个点在范围内，返回这个点的信息
    #     return None  # 如果没有点在范围内，返回None

    # def on_left_click(self,event):
    #     # 假设这个函数已经是Coordinate类的一部分，或者能以某种方式访问self.points_pixel
    #     clicked_point = self.check_click_near_point(event.x, event.y)
    #     if clicked_point:
    #         print(f"Clicked near point: {clicked_point}")
    #         # 这里可以添加更多根据点击点进行的处理逻辑
    #     else:
    #         print("No nearby point was clicked.")

    



In [11]:
def create_triangle(canvas, center_x, center_y, size, **kwargs):

    height = size 
    side_length = height / (3**0.5 / 2)

    x1 = center_x
    y1 = center_y - 2 * height / 3

    x2 = center_x - side_length / 2
    y2 = center_y + height / 3

    x3 = center_x + side_length / 2
    y3 = center_y + height / 3

    canvas.create_polygon(x1, y1, x2, y2, x3, y3, **kwargs)



def create_circle(canvas, center_x, center_y, radius, **kwargs):

    x1 = center_x - radius
    y1 = center_y - radius
    x2 = center_x + radius
    y2 = center_y + radius
    canvas.create_oval(x1, y1, x2, y2, **kwargs)


def draw_star(canvas,center_x, center_y, size, fill):


    points = []
    for i in range(5):
        # 计算外部顶点
        x = center_x + size * math.cos(2 * math.pi * i / 5 - math.pi / 10)
        y = center_y - size * math.sin(2 * math.pi * i / 5 - math.pi / 10)
        points.append((x, y))

        # 计算内部顶点
        x = center_x + size/2 * math.cos(2 * math.pi * i / 5 + math.pi / 10)
        y = center_y - size/2 * math.sin(2 * math.pi * i / 5 + math.pi / 10)
        points.append((x, y))

    canvas.create_polygon(points, outline='black', fill=fill)

In [12]:
def operation_left_click(x,y,unique_labels):
    global current_point_info
    for point in current_point_info:
            if calculate_Euclidean_distance(point['pixel_x'],point['pixel_y'],x,y) < 5.0 :
                current_center = point
                redraw()
                canvas.create_line(0, point['pixel_y'], canvas.winfo_width(), point['pixel_y'], fill='black')
                # 绘制从点击点到画布边缘的竖直线
                canvas.create_line(point['pixel_x'], 0, point['pixel_x'], canvas.winfo_height(), fill='black')
                # 添加更多处理逻辑
                number_of_ticks = 10  # 假设我们想要在每条线上放置10个刻度
                tick_length = 5
                axis_color = 'black'

                # 计算水平线和竖直线的刻度间隔
                gap_x = (500 - 4*30)/ number_of_ticks
                gap_y = (500 - 4*30)/ number_of_ticks

                # 绘制水平线的刻度和标签
                for i in range(number_of_ticks + 1):
                    x = i * gap_x
                    canvas.create_line(x, point['pixel_y'], x, point['pixel_y'] - tick_length, fill=axis_color)
                    # 可以在此处添加对应的标签，例如使用create_text

                # 绘制竖直线的刻度和标签
                for i in range(number_of_ticks + 1):
                    y = i * gap_y
                    canvas.create_line(point['pixel_x'], y, point['pixel_x'] + tick_length, y, fill=axis_color)
                    # 可以在此处添加对应的标签，例如使用create_text

                for p in current_point_info:
                    current_color = '';
                    current_shape = '';
                    if p is current_center:
                        print("same")
                    elif p['value_x']> point['value_x'] and p['value_y']>point['value_y']: #first Quadrant yellow
                        current_color = 'yellow'
                        if p['label'] == unique_labels[0]:
                            create_circle(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'yellow')
                            current_shape = 'circle';
                        elif p['label'] == unique_labels[1]:
                            draw_star(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'yellow')
                            current_shape = 'star';
                        elif p['label'] == unique_labels[2]:
                            create_triangle(canvas,p['pixel_x'],p['pixel_y'],7,fill = 'yellow')
                            current_shape = 'triangle';
                    elif p['value_x']< point['value_x'] and p['value_y']>point['value_y']: #second Quadrant green
                        current_color = 'green'
                        if p['label'] == unique_labels[0]:
                            create_circle(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'green')
                            current_shape = 'circle';
                        elif p['label'] == unique_labels[1]:
                            draw_star(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'green')
                            current_shape = 'star';
                        elif p['label'] == unique_labels[2]:
                            create_triangle(canvas,p['pixel_x'],p['pixel_y'],7,fill = 'green')
                            current_shape = 'triangle';
                    elif p['value_x']< point['value_x'] and p['value_y']<point['value_y']: #3rd Quadrant orange
                        current_color = 'orange'
                        if p['label'] == unique_labels[0]:
                            create_circle(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'orange')
                            current_shape = 'circle';
                        elif p['label'] == unique_labels[1]:
                            draw_star(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'orange')
                            current_shape = 'star';
                        elif p['label'] == unique_labels[2]:
                            create_triangle(canvas,p['pixel_x'],p['pixel_y'],7,fill = 'orange')
                            current_shape = 'triangle';
                    elif p['value_x']> point['value_x'] and p['value_y']<point['value_y']: #4th Quadrant purple
                        current_color = 'purple'
                        if p['label'] == unique_labels[0]:
                            create_circle(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'purple')
                            current_shape = 'circle';
                        elif p['label'] == unique_labels[1]:
                            draw_star(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'purple')
                            current_shape = 'star';
                        elif p['label'] == unique_labels[2]:
                            create_triangle(canvas,p['pixel_x'],p['pixel_y'],7,fill = 'purple')
                            current_shape = 'triangle';
                    # left_click_record.append({'value_x':p['value_x'],
                    #                         'value_y':p['value_y'],
                    #                         'pixel_x':p['pixel_x'],
                    #                         'pixel_y':p['pixel_y'],
                    #                         'label':p['label'],
                    #                         'color':current_color,
                    #                         'shape':current_shape})

In [13]:
import math

def calculate_Euclidean_distance(x1, y1, x2, y2):

    distance_pixel = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance_pixel


def on_left_click(event,unique_labels):
    global current_point_info
    global recent_left_click
    x, y = event.x, event.y
    print(f"Left click at ({x}, {y})")
    print(recent_left_click)
    for point in current_point_info:
        if calculate_Euclidean_distance(point['pixel_x'],point['pixel_y'],x,y) < 5.0 :
            recent_left_click = {'x':x,'y':y}
    operation_left_click(x,y,unique_labels)

    # for point in current_point_info:
    #     if calculate_Euclidean_distance(point['pixel_x'],point['pixel_y'],x,y) < 5.0 :
    #         current_center = point
    #         redraw()
    #         canvas.create_line(0, point['pixel_y'], canvas.winfo_width(), point['pixel_y'], fill='black')
    #         # 绘制从点击点到画布边缘的竖直线
    #         canvas.create_line(point['pixel_x'], 0, point['pixel_x'], canvas.winfo_height(), fill='black')
    #         # 添加更多处理逻辑
    #         number_of_ticks = 10  # 假设我们想要在每条线上放置10个刻度
    #         tick_length = 5
    #         axis_color = 'black'

    #         # 计算水平线和竖直线的刻度间隔
    #         gap_x = (500 - 4*30)/ number_of_ticks
    #         gap_y = (500 - 4*30)/ number_of_ticks

    #         # 绘制水平线的刻度和标签
    #         for i in range(number_of_ticks + 1):
    #             x = i * gap_x
    #             canvas.create_line(x, point['pixel_y'], x, point['pixel_y'] - tick_length, fill=axis_color)
    #             # 可以在此处添加对应的标签，例如使用create_text

    #         # 绘制竖直线的刻度和标签
    #         for i in range(number_of_ticks + 1):
    #             y = i * gap_y
    #             canvas.create_line(point['pixel_x'], y, point['pixel_x'] + tick_length, y, fill=axis_color)
    #             # 可以在此处添加对应的标签，例如使用create_text

    #         for p in current_point_info:
    #             current_color = '';
    #             current_shape = '';
    #             if p is current_center:
    #                 print("same")
    #             elif p['value_x']> point['value_x'] and p['value_y']>point['value_y']: #first Quadrant yellow
    #                 current_color = 'yellow'
    #                 if p['label'] == unique_labels[0]:
    #                     create_circle(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'yellow')
    #                     current_shape = 'circle';
    #                 elif p['label'] == unique_labels[1]:
    #                     draw_star(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'yellow')
    #                     current_shape = 'star';
    #             elif p['value_x']< point['value_x'] and p['value_y']>point['value_y']: #second Quadrant green
    #                 current_color = 'green'
    #                 if p['label'] == unique_labels[0]:
    #                     create_circle(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'green')
    #                     current_shape = 'circle';
    #                 elif p['label'] == unique_labels[1]:
    #                     draw_star(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'green')
    #                     current_shape = 'star';
    #             elif p['value_x']< point['value_x'] and p['value_y']<point['value_y']: #3rd Quadrant orange
    #                 current_color = 'orange'
    #                 if p['label'] == unique_labels[0]:
    #                     create_circle(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'orange')
    #                     current_shape = 'circle';
    #                 elif p['label'] == unique_labels[1]:
    #                     draw_star(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'orange')
    #                     current_shape = 'star';
    #             elif p['value_x']> point['value_x'] and p['value_y']<point['value_y']: #4th Quadrant purple
    #                 current_color = 'purple'
    #                 if p['label'] == unique_labels[0]:
    #                     create_circle(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'purple')
    #                     current_shape = 'circle';
    #                 elif p['label'] == unique_labels[1]:
    #                     draw_star(canvas,p['pixel_x'],p['pixel_y'],5,fill = 'purple')
    #                     current_shape = 'star';
    #             left_click_record.append({'value_x':p['value_x'],
    #                                       'value_y':p['value_y'],
    #                                       'pixel_x':p['pixel_x'],
    #                                       'pixel_y':p['pixel_y'],
    #                                       'label':p['label'],
    #                                       'color':current_color,
    #                                       'shape':current_shape})
                    


def on_right_click(event,unique_labels):
    x, y = event.x, event.y
    current_center = []
    points_distance = []
    global last_right_click_point
    global last_left_click
    #detect right clicking a point
    for point in current_point_info:
        if calculate_Euclidean_distance(point['pixel_x'],point['pixel_y'],x,y) < 5.0 :
            current_center = point

    if current_center: #click the data point
        redraw()
        operation_left_click(recent_left_click['x'],recent_left_click['y'],unique_labels)
        if current_center != last_right_click_point:        #first time click
            last_right_click_point = current_center
            for p in current_point_info:    #执行查找最近的10个点并highlight
                points_distance.append({'pixel_x':p['pixel_x'],'pixel_y':p['pixel_y'],'dis2centre':calculate_Euclidean_distance(current_center['pixel_x'],current_center['pixel_y'],p['pixel_x'],p['pixel_y'])})
            
                    # sort by distance
            points_distance.sort(key=lambda item: item['dis2centre'])
            
            # 选取距离最小的前十个点
            closest_points = points_distance[1:11]
             # 对这些点进行绘制
            for point in closest_points:
                create_circle(canvas,point['pixel_x'], point['pixel_y'], 7, fill = 'pink')
        elif current_center==last_right_click_point:
            if recent_left_click == []:
                redraw()
                last_right_click_point = []
            else:
                operation_left_click(recent_left_click['x'],recent_left_click['y'],unique_labels)
                last_right_click_point = []





def on_r_key_press(event):
    global recent_left_click
    global last_right_click_point
    
    recent_left_click = []
    last_right_click_point = []
    redraw()


def redraw():
    canvas.delete("all")
    coordinate = Coordinate(width=500, height=500, margin=30, canvas=canvas)
    coordinate.create_axes_with_ticks()
    coordinate.draw_points()

highlight = False
current_left_click = ''
# Create Tkinter window
window = tk.Tk()
window.title("Axes with Ticks")

# Set window size
window.geometry('600x600')

# Create Canvas widget
width = 500
height = 500
canvas = tk.Canvas(window, width=width, height=height)
canvas.pack()



#canvas.bind("<Button-1>", lambda event, instance=my_instance: left_click(event, instance))

coordinate = Coordinate(width=500, height=500, margin=30, canvas=canvas)
coordinate.create_axes_with_ticks()
coordinate.draw_points()

current_point_info = coordinate.get_All_Points_Info()
left_click_record = []
recent_left_click = {'x': None, 'y': None}
last_right_click_point = []

canvas.bind("<Button-1>", lambda event, unique_labels = coordinate.unique_labels: on_left_click(event,unique_labels) )
canvas.bind("<Button-3>", lambda event, unique_labels = coordinate.unique_labels: on_right_click(event,unique_labels))
window.bind('<KeyPress-r>', on_r_key_press)










# Run the window's main loop
window.mainloop()


Coordinate System Details:
  Width (w): 500
  Height (h): 500
  Margin (m): 30
  X Axis Range: -60 to 60
  Y Axis Range: -60 to 60
  Value X Low (value_x_l): -52
  Value X High (value_x_h): 55
  Value Y Low (value_y_l): -55
  Value Y High (value_y_h): 54
  Number of Labels (number_labels): ['a' 'b' 'c']


In [14]:
if '':
    print("a")
